# Notebook pour l'entrainement des modèles 


## Importation des modules

In [1]:
import tensorflow as tf
import numpy as np
import os  
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
import numpy as np
import keras
from tensorflow.keras.applications import VGG16
import random
from tensorflow.keras.preprocessing import image
import cv2
from PIL import Image
import pandas as pd
import pathlib
from tensorflow.keras.applications import EfficientNetB0
import tensorflow_addons as tfa

### On choisit le batch_size que l'on souhaite utililser
Nous avons constater que pour avoir un temps d'execution minimal, le batch_size dépendait principalement des machines sur lequel on entreprend l'entrainement. 

In [2]:
batch_size=512

## Importation du csv 
Le csv "Afternotfungi2.csv" contient tous les chemins des photos présentes dans le dossier esperons
Si vous voulez faire fonctionner le notebook, il vous faudra avoir télécharger les images et indiquer le dossier dans base_dir
Ensuite la colonne path de df contiendra vos chemins personalisés jusqu'a vos photos.


In [17]:
import pandas as pd
import pathlib

# Chargement du fichier
df = pd.read_csv ("Champyseed.csv",index_col=0)

#Création d'une nouvelle variable contenant l'arborescence sur le HDD des fichiers images
df["path"] = '\\'+df["order"]+'\\'+df["family"]+'\\'+df["genus"]+'\\'+df["species"]+'\\'+'im'+df.notreid.astype('str')+".jpg"

# On ajoute le chemin où se trouve l'arborescence précédente (peut varier selon les personnes)
base_dir = r'C:\Users\baugn\Mush\esperons'
df["path"] = base_dir+df["path"]

C:\Users\baugn\anaconda3\envs\gputest\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Choix des données pour l'entrainement
Il vous faut choisir le nombre de classe, le type de classe et le nombre de photos par classe

In [ ]:
nombre_classe=64
types="genus"     #décommenter la ligne que vous souhaitez
#type="species"
nombre_de_photos_par_classe=8000

In [19]:
#La cellule suivante va creer un dataframe data qui va contenir les lignes du dataframe df avec les caractéristiques que vous avez choisies
#précédemment: Dans l'exemple pré-enregistré data contiendra 8000 lignes pour chaque genus. Le choix des genus est choisi parmis tous les genus
#triés par ordre décroissant de photos disponibles. 
#Si jamais il n'y a pas assez de photos dans un genus , alors la fonction prendra tout ce qu'elle peut au sein de ce genus
# Dans l'exemple toutes les classes ont plus de 8000 photos et elles ont donc toutes 8000 photos.

def creation_de_ma_data(nombre_classe):
    liste_classe=df[types].value_counts().index.tolist()
    if nombre_classe>len(df[types].unique()):
        print("Pas assez de classes dans le dataset actuel (",len(df[types].unique()),")")
        return
    
    c=pd.DataFrame()
    for i in range(0,nombre_classe):
        if len(df[df[types]==liste_classe[i]])>nombre_de_photos_par_classe:
            cplus=df[df[types]==liste_classe[i]].sample(nombre_de_photos_par_classe)
        else:
            cplus=df[df[types]==liste_classe[i]].sample(len(df[df[types]==liste_classe[i]]))
        c=pd.concat([c,cplus])
    return c
data=creation_de_ma_data(nombre_classe)

# Différenciation des deux méthodes utilisées -- Méthode avec ImageDataGenerator
Nous avons utilisé deux types d'entrainement. Le premier utilise le module ImageDataGenrator qui permet de se simplifier beaucoup la vie en terme de code. ImageFataGenerator nous permet de créer un genérateur d'image qui peut se "nourir" d'un dataframe en lui indiquanr les colonnes du chemin des photos, et celui de la target.
ImageDataGenerator permet aussi de faire de la data augmentation. Ici nous n'utilisons que 3 fonctions d'augmentation.
rotation_range
zoom_range
brightness_range

## Découpage en train test de notre dataframe

In [20]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size = 0.2)

In [21]:
from tensorflow.keras.applications.resnet import preprocess_input

# Création de deux générateurs d'images différent, un pour la partie Train et un pour la partie Test
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   rotation_range = 180,
                                   #width_shift_range = 0.2,
                                   #height_shift_range = 0.2,
                                   #shear_range = 0.2,
                                   zoom_range = 0.2,
                                   brightness_range = [0.9,1.1])
                                   #horizontal_flip = False)


test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [23]:

#On indique avec la méthode flow_from_dataframe les différentes caractéristiques utiles de notre dataframe
train_generator = train_datagen.flow_from_dataframe(dataframe=data_train,
                                                    directory = "",
                                                    x_col = "path",
                                                    y_col = types,
                                                    batch_size = batch_size,
                                                    class_mode = 'sparse', #"raw", #"'binary'
                                                    target_size = (224, 224))

# Validation avec les images de test sur le HDD
validation_generator = test_datagen.flow_from_dataframe(dataframe=data_test,
                                                        directory="",
                                                        x_col = "path",
                                                        y_col = types,
                                                        batch_size = batch_size,
                                                        class_mode = 'sparse',
                                                        target_size = (224, 224))

Found 409600 validated image filenames belonging to 64 classes.
Found 102400 validated image filenames belonging to 64 classes.


## Définition des callbacks
Les callbacks sont des outils qui permettent d'effectuer des contrôles au cours de l'entrainement
Nous en utilisons 3:
- checkpoint : permet d'enregistrer le modèle, le modèle est sauvegardé à chaque fin d'epoch quand la valeur "monitor" s'est améliorée
- early : permet d'arreter le modèle quand la valeur "monitor" ne s'améliore pas pendant le nombre d'epoch "patience"
- reduce_lr : permet de réduire(par la valeur "factor") le learning rate quand la valeur monitor ne s'est pas améliorée pendant le nombre d'epoch "patience"

In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("res50imdatagen.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq="epoch")

early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=6, 
                      verbose=1, 
                      mode='auto')


reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2,
                              patience=3,
                              cooldown=1,
                             verbose=1)

## Importation du modèle pré-entrainé (Transfert Learning)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50

## Création de la couche de classification, des métriques utilisées et  compilation

In [26]:
base_model = ResNet50(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')


for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units = 1024, activation = 'relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units = nombre_classe, activation = 'softmax'))

model.compile(optimizer = "adam", #optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), 
                   loss = 'sparse_categorical_crossentropy', #loss = 'binary_crossentropy', 
                   metrics = ['acc',tf.keras.metrics.SparseTopKCategoricalAccuracy(
    k=5, name='top_5'),tf.keras.metrics.SparseTopKCategoricalAccuracy(
    k=3, name='top_3')])

## Entrainement du modèle

In [ ]:
history = model.fit(train_generator, 
                          validation_data = validation_generator,
                    epochs=50, 
                    steps_per_epoch = len(data_train) // batch_size,
                         validation_steps= len(data_test) // batch_size,
                    callbacks=[checkpoint, early, reduce_lr])

Epoch 1/50
800/800 [==============================] - 4282s 5s/step - loss: 2.3558 - acc: 0.3706 - top_5: 0.6844 - top_3: 0.5835 - val_loss: 1.8056 - val_acc: 0.4998 - val_top_5: 0.8003 - val_top_3: 0.7123

Epoch 00001: val_acc improved from -inf to 0.49980, saving model to res50imdatagen.h5


C:\Users\baugn\anaconda3\envs\gputest\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
800/800 [==============================] - 5163s 6s/step - loss: 2.0031 - acc: 0.4527 - top_5: 0.7624 - top_3: 0.6701 - val_loss: 1.7277 - val_acc: 0.5201 - val_top_5: 0.8161 - val_top_3: 0.7323

Epoch 00002: val_acc improved from 0.49980 to 0.52009, saving model to res50imdatagen.h5
Epoch 3/50
800/800 [==============================] - 5032s 6s/step - loss: 1.9050 - acc: 0.4758 - top_5: 0.7819 - top_3: 0.6925 - val_loss: 1.6490 - val_acc: 0.5401 - val_top_5: 0.8265 - val_top_3: 0.7470

Epoch 00003: val_acc improved from 0.52009 to 0.54011, saving model to res50imdatagen.h5
Epoch 4/50
800/800 [==============================] - 4603s 6s/step - loss: 1.8503 - acc: 0.4889 - top_5: 0.7915 - top_3: 0.7041 - val_loss: 1.6126 - val_acc: 0.5494 - val_top_5: 0.8338 - val_top_3: 0.7561

Epoch 00004: val_acc improved from 0.54011 to 0.54937, saving model to res50imdatagen.h5
Epoch 5/50
800/800 [==============================] - 4343s 5s/step - loss: 1.8085 - acc: 0.5004 - top_5: 0.7998

Epoch 31/50
800/800 [==============================] - 4535s 6s/step - loss: 1.4363 - acc: 0.5908 - top_5: 0.8619 - top_3: 0.7916 - val_loss: 1.3836 - val_acc: 0.6099 - val_top_5: 0.8678 - val_top_3: 0.8008

Epoch 00031: val_acc did not improve from 0.61004
Epoch 32/50
800/800 [==============================] - 4754s 6s/step - loss: 1.4317 - acc: 0.5928 - top_5: 0.8625 - top_3: 0.7923 - val_loss: 1.3827 - val_acc: 0.6101 - val_top_5: 0.8681 - val_top_3: 0.8008

Epoch 00032: val_acc improved from 0.61004 to 0.61009, saving model to res50imdatagen.h5
Epoch 33/50
800/800 [==============================] - 4707s 6s/step - loss: 1.4281 - acc: 0.5936 - top_5: 0.8633 - top_3: 0.7932 - val_loss: 1.3802 - val_acc: 0.6102 - val_top_5: 0.8684 - val_top_3: 0.8013

Epoch 00033: val_acc improved from 0.61009 to 0.61021, saving model to res50imdatagen.h5
Epoch 34/50
800/800 [==============================] - 4765s 6s/step - loss: 1.4257 - acc: 0.5934 - top_5: 0.8636 - top_3: 0.7938 - val_loss: 1.3802

In [1]:
model.summary()


NameError: name 'model' is not defined

## Graphique montrant l'évolution de la précision et la perte en fonction des epochs

In [2]:
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')

plt.subplot(122)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['val_top_5'])
plt.plot(history.history['val_top_3'])
plt.plot(history.history['val_top_5'])
plt.plot(history.history['val_top_3'])
plt.title('Model acc by epoch')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')
plt.show()

NameError: name 'plt' is not defined

## Différenciation des deux méthodes utilisées -- Méthode sans ImageDataGenerator

le modèle est le même mais nous devons efféctué beaucoup plus de travail afin de préparer les données à être entrainées.


# Création d'un array avec la liste des types


In [3]:
CLASS_NAMES=np.array(data["genus"].unique().tolist())

NameError: name 'np' is not defined

In [4]:
CLASS_NAMES

NameError: name 'CLASS_NAMES' is not defined

# Découpage du Train Test. Pour pouvoir utilisé Tensorslices, nous ne devont garder qu'une seule colonne. 

In [5]:
from sklearn.model_selection import train_test_split
train,test= train_test_split(data["path"],test_size=0.2,random_state=42,shuffle=True)

NameError: name 'data' is not defined

## Définition des fonctions pour mettre en place les données


In [7]:
if types=="genus":
    k=-3
elif types=="species":
    k=-2
    
def parse_image(filename):     #cette fonction à partir du chemin du fichier lit l'image et en extrait aussi le label 
    parts = tf.strings.split(filename, '\\')    #Le label est ici un array de boolén qui a sa valeur True sur l'index qui
    label = CLASS_NAMES==parts[k]        #correspond à la position du types trouvé dans CLASS_NAMES
    image = tf.io.read_file(filename)   #l'image est aussi décodé de jpeg
    image = tf.image.decode_jpeg(image)
    
   
    
    image = tf.image.convert_image_dtype(image, tf.float32)
    
        

    return image, label

NameError: name 'types' is not defined

In [8]:
def augment(image):   #Fonction de data augmentation
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    
    # Flips
    if p_spatial >= .2:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º

    return image

In [9]:
def augment_image_train(image, label):   #Fonction augment pour le train
    
    #image=tf.image.random_flip_left_right(image)
    #image=tf.image.random_flip_up_down(image)
     
    image=augment(image)
    image=tf.image.per_image_standardization(image)
    image=tf.image.random_brightness(image, max_delta=0.1)
    image=tf.keras.applications.nasnet.preprocess_input(image)
    #image=tf.image.random_zoom(image, zoom_range=(0.1,0.2),  fill_mode='nearest')
    return image, tf.cast(label, tf.float32)     #le label booléen est transformé en array de float 

In [10]:
def augment_image_test(image, label):      #idem pour le test
    
    #image=tf.image.random_flip_left_right(image)
    #image=tf.image.random_flip_up_down(image)
     
    #image=augment(image)
    image=tf.image.per_image_standardization(image)
    #image=tf.image.random_brightness(image, max_delta=0.1)
    #image=tf.image.random_zoom(image, zoom_range=(0.1,0.2),  fill_mode='nearest')
    image=tf.keras.applications.nasnet.preprocess_input(image)
    return image, tf.cast(label, tf.float32)


## Création des deux pipelines par lequel les deux deux datasets doivent passer pour rentrer dans le modèle


In [11]:
def create_dataset_train(file_list,shuffle_buffer_size=1000):       #pipeline pour le train
  
    ds=tf.data.Dataset.from_tensor_slices(file_list)
  
    ds=ds.shuffle(buffer_size=len(file_list))

    ds=ds.map(parse_image,num_parallel_calls=tf.data.AUTOTUNE)

    ds=ds.map(augment_image_train,num_parallel_calls=tf.data.AUTOTUNE)
  

    ds=ds.repeat()

    ds=ds.batch(batch_size)

    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  
    return ds

In [12]:
def create_dataset_test(file_list,shuffle_buffer_size=1000):    #pipeline pour le test
  
    ds=tf.data.Dataset.from_tensor_slices(file_list)
  
    ds=ds.shuffle(buffer_size=len(file_list))

    ds=ds.map(parse_image,num_parallel_calls=tf.data.AUTOTUNE)

    ds=ds.map(augment_image_test,num_parallel_calls=tf.data.AUTOTUNE)
  

    ds=ds.repeat()

    ds=ds.batch(batch_size)

    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  
    return ds

## Création des deux datasets

In [ ]:
train_ds=create_dataset_train(train)                   
test_ds=create_dataset_test(test)

## Entrainement


In [13]:
history = model.fit(train_ds,
                    epochs=50, 
                    steps_per_epoch = len(train) // batch_size,
                         validation_steps= len(test) // batch_size,
                    callbacks=[checkpoint, early, reduce_lr],
                    validation_data=test_ds)


NameError: name 'model' is not defined

## Graphique montrant l'évolution de la précision et la perte en fonction des epochs

In [14]:
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')

plt.subplot(122)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['val_top_5'])
plt.plot(history.history['val_top_3'])
plt.plot(history.history['val_top_5'])
plt.plot(history.history['val_top_3'])
plt.title('Model acc by epoch')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')
plt.show()

NameError: name 'plt' is not defined